In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
! pip install -q kaggle

In [ ]:
files.upload() #upload kaggle.json file

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minghuaq","key":"663f952bdd8fd276710243a49d89fedd"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
def downloadKaggleData(link):
  name = (link.split(' ')[-1]).split('/')[-1]
  ! {link}
  ! mkdir {name}
  ! unzip {name}.zip -d {name}

In [ ]:
downloadKaggleData("kaggle competitions download -c digit-recognizer")

 98% 15.0M/15.3M [00:00<00:00, 79.3MB/s]
100% 15.3M/15.3M [00:00<00:00, 75.3MB/s]
Archive:  digit-recognizer.zip
  inflating: digit-recognizer/sample_submission.csv  
  inflating: digit-recognizer/test.csv  
  inflating: digit-recognizer/train.csv  


In [ ]:
from torch.utils.data import Dataset, DataLoader, Sampler

class TrainDataset(Dataset):
    def __init__(self, dataLink):
        self.data = pd.read_csv(dataLink).values
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, ind):
        x = self.data[ind][1:] / 255.0
        y = self.data[ind][0]
        return x, y
class TestDataset(TrainDataset):
    def __getitem__(self, ind):
        x = self.data[ind] / 255.0
        return x

train_set = TrainDataset("digit-recognizer/train.csv")
test_set  = TestDataset("digit-recognizer/test.csv")

print(train_set[0])
batch_size = 512
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)

for X, y in train_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# print(train_set[3][1])

number_of_class = max(train_set.data[:,0]) - min(train_set.data[:,0]) + 1
print(f"Number of classes: {number_of_class}")

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    

In [ ]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} device')

class MLP(nn.Module):
  def __init__(self) -> None:
    super(MLP, self).__init__()
    inputlayer = [28*28]
    hiddenlayer = []
    outputlayer = number_of_class
    hiddenlayer = inputlayer + hiddenlayer

    self.layers = nn.Sequential()
    for i in range(len(hiddenlayer)-1):
      self.layers.add_module(f'linear_{i+1}', nn.Linear(hiddenlayer[i], hiddenlayer[i+1]))
      self.layers.add_module(f'relu_{i+1}', nn.ReLU())
    self.layers.add_module(f'linear_last', nn.Linear(hiddenlayer[-1], outputlayer))

  def forward(self, x):
    x = x.float()
    return self.layers(x)

model = MLP().to(device)
print(model)

Using cpu device
MLP(
  (layers): Sequential(
    (linear_last): Linear(in_features=784, out_features=10, bias=True)
  )
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  model.train()

  num_batches = len(dataloader)
  num_items = len(dataloader.dataset)

  total_loss = 0
  total_correct = 0

  losses = []
  for batch_num, (X, y) in enumerate(dataloader):
    optimizer.zero_grad()
    X, y = X.to(device), y.to(device)

    output = model(X)

    loss = loss_fn(output, y)

    total_loss += loss
    total_correct += (torch.argmax(output,dim = 1)== y).sum().item()
    loss.backward()
    losses.append(loss.item())
    optimizer.step()

  train_loss = total_loss/num_batches
  accuracy = total_correct/num_items

  print(f"Average loss: {train_loss:7f}, accuracy: {accuracy:.2%}")



In [ ]:
epochs = 10
for epoch in range(epochs):
    print(f"Training epoch: {epoch+1}")
    train(train_loader, model, criterion, optimizer)

Training epoch: 1
Average loss: 0.308639, accuracy: 91.58%
Training epoch: 2
Average loss: 0.302819, accuracy: 91.75%
Training epoch: 3
Average loss: 0.300289, accuracy: 91.82%
Training epoch: 4
Average loss: 0.291997, accuracy: 92.01%
Training epoch: 5
Average loss: 0.285589, accuracy: 92.09%
Training epoch: 6
Average loss: 0.290225, accuracy: 92.20%
Training epoch: 7
Average loss: 0.278050, accuracy: 92.29%
Training epoch: 8
Average loss: 0.276614, accuracy: 92.34%
Training epoch: 9
Average loss: 0.272285, accuracy: 92.46%
Training epoch: 10
Average loss: 0.273971, accuracy: 92.47%
